## Waves in space-time

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns
from scipy.integrate import solve_ivp
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_style("whitegrid")
from IPython.display import HTML
from matplotlib import animation
from bspline import snake_bspline
%matplotlib inline

# Don't tinker, or do
#%matplotlib nbagg
# from matplotlib import rcParams
#rcParams['font.family']='sans-serif' 
#rcParams('font', serif='Helvetica Neue') 
# rcParams['text.usetex']= True 
#rcParams.update({'font.size': 22})

In [ ]:
class MplColorHelper:

    def __init__(self, cmap_name, start_val, stop_val):
        self.cmap_name = cmap_name
        self.cmap = plt.get_cmap(cmap_name)
        self.norm = mpl.colors.Normalize(vmin=start_val, vmax=stop_val)
        self.scalarMap = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

    def get_rgb(self, val):
        return self.scalarMap.to_rgba(val)

In [ ]:
def animator(t_fig, t_renderer, t_data, t_color_by, t_cmap='RdBu'):
    """ Access to the animate class from matplotlib
    """
    COL = MplColorHelper(t_cmap, 0.0, 1.0)
    
    n_points = t_data.shape[1]
    minmax_x = np.amax(np.abs(t_data[0, :, :]))
    minmax_y = np.amax(np.abs(t_data[1, :, :]))

    coloring = t_color_by(t_data)
    max_color = np.amax(coloring)
    min_color = np.amin(coloring)
    
    # animation function. This is called sequentially
    def animate_in(i):
        t_renderer.clear()

        loc_color = ( t_color_by(t_data[: , :, i]) - min_color)/(max_color - min_color)

#         for seg in range(n_points-1):
#             t_renderer.plot([t_data[0, seg, i], t_data[0, seg + 1, i]], [t_data[1, seg, i], t_data[1, seg + 1, i]], c=COL.get_rgb(loc_color[seg]))

        t_renderer.plot(t_data[0, :, i], t_data[1, :, i])

        # Plot centreline
        t_renderer.plot([0.0, 1.0], [0.0, 0.0],'k--')

        t_renderer.set_xlim(-0.05, minmax_x)
        t_renderer.set_ylim(-minmax_y, minmax_y)
        t_renderer.set_xlabel(r'$x$')
        t_renderer.set_ylabel(r'$y$')
        t_renderer.set_aspect('auto')

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(t_fig, animate_in, frames=100, interval=5)

    return anim

In [ ]:
spline_coeffs = np.array([1.0, 2.0, 1.0])
wave_spline = snake_bspline(spline_coeffs)

# Kinematics

In [ ]:
non_dim_cline = np.linspace(0.0, 1.0, 51, endpoint=True)
non_dim_time = np.linspace(0.0, 1.0, 101, endpoint=True)

# Standing wave

In [ ]:
sp_k = 1
#sp_k = 3
#sp_k = 2.5

#y_prefac = 0.01
#y_prefac = 0.1
y_prefac = 0.15

y_t = np.sin(2.*np.pi*non_dim_time.reshape(-1, 1))
y_s = np.sin(2.*np.pi*sp_k*non_dim_cline.reshape(1, -1))
y = y_prefac*y_t*y_s

x = 0.0*y_t*y_s

In [ ]:
def run_standing_wave():
    for i in range(non_dim_time.shape[0]):
        def dy_ds(s, y):
            return y_prefac*2.*np.pi*sp_k*np.cos(2.*np.pi*sp_k*s)*y_t[i]

        def dx_ds(s, y):
            return np.sqrt(1.0 - dy_ds(s,y)**2)

        x_s = solve_ivp(dx_ds, (0.0, 1.0), [0.0], t_eval = non_dim_cline)

        # Gives (npoints, 1) back out
        x[i] = x_s.y.reshape(-1, )

    # In (time x shape x loc)
    data_vec = np.dstack((x,y))

    # In (loc x shape x time)
    data_vec = data_vec.T
    
    return data_vec

In [ ]:
data_vec = run_standing_wave()

def colorby(t_data):
    return t_data[1]

fig, ax = plt.subplots(1,1, figsize=(10, 2))
anim = animator(fig, ax, data_vec, colorby)

In [ ]:
# anim.save('verlet.mp4', fps=30, 
#            extra_args=['-vcodec', 'h264', 
#                        '-pix_fmt', 'yuv420p'])

In [ ]:
t_index = slice(1,50)
ax.clear()
ax.plot(data_vec[0, : , t_index], data_vec[1, : , t_index])
ax.set_aspect('auto')
fig

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
def func(s):
    # return 1 + 0.0*s, 0.0 + 0.0*s
    # return s, 1 + 0.0*s
    # return s**2, 2*s
    return wave_spline(s), (wave_spline.derivative())(s)

# Change only for the case of (1., 0.0) of func
sp_k = 1
#sp_k = 3
#sp_k = 2.5

y_prefac = 0.01
#y_prefac = 0.1
#y_prefac = 0.15

# y_t = np.sin(2.*np.pi*non_dim_time.reshape(-1, 1))
# y_s = np.sin(2.*np.pi*sp_k*non_dim_cline.reshape(1, -1))
f, _ = func(non_dim_cline.reshape(1, -1))
y = y_prefac * f * np.sin(2.*np.pi*(-non_dim_time.reshape(-1, 1) + sp_k*non_dim_cline.reshape(1, -1)))

x = 0.0*y_t*y_s

In [ ]:
def run_ramped_travelling_wave():
    for i in range(non_dim_time.shape[0]):
        
        def dy_ds(s, y):
            f, fdash = func(s)
            return y_prefac*(2.*np.pi*sp_k*np.cos(2.*np.pi*(-non_dim_time[i] + sp_k*s))*f + fdash*np.sin(2.*np.pi*(non_dim_time[i] + sp_k*s)))

        def dx_ds(s, y):
            return np.sqrt(1.0 - dy_ds(s,y)**2)

        x_s = solve_ivp(dx_ds, (0.0, 1.0), [0.0], t_eval = non_dim_cline)

        # Gives (npoints, 1) back out
        x[i] = x_s.y.reshape(-1, )

    # In (time x shape x loc)
    data_vec = np.dstack((x,y))

    # In (loc x shape x time)
    data_vec = data_vec.T
    
    return data_vec

In [ ]:
data_vec = run_ramped_travelling_wave()

def colorby(t_data):
    return t_data[1]

fig, ax = plt.subplots(1,1, figsize=(10, 2))
anim = animator(fig, ax, data_vec, colorby)
HTML(anim.to_jshtml())